In [2]:
from __future__ import division
from api import git_access,api_access
from git_understand import git_understand_v1 as git_understand
from git_log import git2repo
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import networkx as nx
import re
from git_log import git2data,git_commit_info,release_mine,project_url_mine
import threading
from threading import Barrier
from multiprocessing import Queue
from os.path import dirname as up
import os
import platform
from commit_guru.cas_manager_v1 import *

In [28]:
if platform.system() == 'Darwin' or platform.system() == 'Linux':
    data_path = os.getcwd() + '/git_list.csv'
else:
    data_path = os.getcwd() + '\\Test_projects.csv'
code_path = os.getcwd()
project_list = pd.read_csv(data_path)
for i in range(project_list.shape[0]):
    try:
        print("I am here")
        understand_source = []
        last_analyzed = None
        access_token = project_list.loc[i,'access_token']
        repo_owner = project_list.loc[i,'repo_owner']
        source_type = project_list.loc[i,'source_type']
        git_url = project_list.loc[i,'git_url']
        api_base_url = project_list.loc[i,'api_base_url']
        repo_name = project_list.loc[i,'repo_name']
        repo_lang = project_list.loc[i,'lang']
        #git_data = git2data.git2data(access_token,repo_owner,source_type,git_url,api_base_url,repo_name)
    #     git_data = project_url_mine.git2data(access_token,repo_owner,source_type,git_url,api_base_url,repo_name)
    #     git_data.create_data()
        file = pd.read_csv('/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/' + repo_name + '.csv')
        commit_nums = file.shape[0]
        buggy_commits = file[file['contains_bug'] == True].shape[0]
        project_list.loc[i,'commits'] = commit_nums
        project_list.loc[i,'defective'] = buggy_commits
#         matrix = get_matrix.get_defective_pair_metrics()
    except Exception as e:
        print("Exception occured for ",project_list.loc[i,'git_url'])
        print(e)
        continue

I am here
I am here
Exception occured for  https://github.com/consulo/consulo
[Errno 2] File b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/consulo.csv' does not exist: b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/consulo.csv'
I am here
Exception occured for  https://github.com/BashSupport/BashSupport
[Errno 2] File b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/BashSupport.csv' does not exist: b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/BashSupport.csv'
I am here
I am here
Exception occured for  https://github.com/apple/servicetalk
'contains_bug'
I am here
I am here
I am here
I am here
I am here
I am here
I am here
I am here
Exception occured for  https://github.com/oracle/fastr
'contains_bug'
I am here
I am here
I am here
I am here
Exception occured for  https://github.com/reactiverse/es4x
'contains_bug'
I am here
I am here
Exception occured for  https://github

I am here
Exception occured for  https://github.com/vmware/admiral
[Errno 2] File b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/admiral.csv' does not exist: b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/admiral.csv'
I am here
Exception occured for  https://github.com/gaul/modernizer-maven-plugin
[Errno 2] File b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/modernizer-maven-plugin.csv' does not exist: b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/modernizer-maven-plugin.csv'
I am here
Exception occured for  https://github.com/willowtreeapps/Hyperion-Android
'contains_bug'
I am here
Exception occured for  https://github.com/anvil-ui/anvil
'contains_bug'
I am here
I am here
Exception occured for  https://github.com/f2prateek/dart
'contains_bug'
I am here
I am here
Exception occured for  https://github.com/puniverse/capsule
'contains_bug'
I am here
Exception occured fo

Exception occured for  https://github.com/opencypher/openCypher
'contains_bug'
I am here
I am here
I am here
Exception occured for  https://github.com/tcoulter/jockeyjs
[Errno 2] File b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/jockeyjs.csv' does not exist: b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/jockeyjs.csv'
I am here
I am here
I am here
Exception occured for  https://github.com/redpen-cc/redpen
'contains_bug'
I am here
Exception occured for  https://github.com/wro4j/wro4j
[Errno 2] File b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/wro4j.csv' does not exist: b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/wro4j.csv'
I am here
Exception occured for  https://github.com/joltup/react-native-threads
[Errno 2] File b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/react-native-threads.csv' does not exist: b'/Users/suvodeepmajumder/Doc

I am here
Exception occured for  https://github.com/vanilla-music/vanilla
[Errno 2] File b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/vanilla.csv' does not exist: b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/vanilla.csv'
I am here
Exception occured for  https://github.com/ryantenney/metrics-spring
[Errno 2] File b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/metrics-spring.csv' does not exist: b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/metrics-spring.csv'
I am here
Exception occured for  https://github.com/ram-on/SkyTube
[Errno 2] File b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/SkyTube.csv' does not exist: b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/SkyTube.csv'
I am here
Exception occured for  https://github.com/opensagres/xdocreport
[Errno 2] File b'/Users/suvodeepmajumder/Documents/AI4SE/Dat

[Errno 2] File b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/TarsosDSP.csv' does not exist: b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/TarsosDSP.csv'
I am here
Exception occured for  https://github.com/spring-projects/spring-retry
[Errno 2] File b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/spring-retry.csv' does not exist: b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/spring-retry.csv'
I am here
Exception occured for  https://github.com/bpellin/keepassdroid
[Errno 2] File b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/keepassdroid.csv' does not exist: b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/keepassdroid.csv'
I am here
Exception occured for  https://github.com/dain/leveldb
[Errno 2] File b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/leveldb.csv' does not exist: b'/

...

I am here
Exception occured for  https://github.com/jersey/jersey
[Errno 2] File b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/jersey.csv' does not exist: b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/jersey.csv'
I am here
Exception occured for  https://github.com/lgvalle/Material-Animations
[Errno 2] File b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/Material-Animations.csv' does not exist: b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/Material-Animations.csv'
I am here
Exception occured for  https://github.com/react-native-community/react-native-camera
[Errno 2] File b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/react-native-camera.csv' does not exist: b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/react-native-camera.csv'
I am here
Exception occured for  https://github.com/elasticjob/elastic-job-lite


I am here
Exception occured for  https://github.com/eclipse/microprofile
'contains_bug'
I am here
Exception occured for  https://github.com/GumTreeDiff/gumtree
[Errno 2] File b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/gumtree.csv' does not exist: b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/gumtree.csv'
I am here
I am here
I am here
Exception occured for  https://github.com/flaxsearch/luwak
[Errno 2] File b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/luwak.csv' does not exist: b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/luwak.csv'
I am here
Exception occured for  https://github.com/cobbzilla/s3s3mirror
[Errno 2] File b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/s3s3mirror.csv' does not exist: b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/s3s3mirror.csv'
I am here
Exception occured for  https://gi

I am here
I am here
I am here
Exception occured for  https://github.com/google/copybara
[Errno 2] File b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/copybara.csv' does not exist: b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/copybara.csv'
I am here
I am here
I am here
Exception occured for  https://github.com/konifar/droidkaigi2016
'contains_bug'
I am here
Exception occured for  https://github.com/bartdag/py4j
[Errno 2] File b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/py4j.csv' does not exist: b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/py4j.csv'
I am here
Exception occured for  https://github.com/javafx-maven-plugin/javafx-maven-plugin
'contains_bug'
I am here
Exception occured for  https://github.com/google/conscrypt
[Errno 2] File b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/conscrypt.csv' does not exist: b'/Users/suvodeepmaj

I am here
Exception occured for  https://github.com/jcabi/jcabi-aspects
[Errno 2] File b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/jcabi-aspects.csv' does not exist: b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/jcabi-aspects.csv'
I am here
Exception occured for  https://github.com/bardsoftware/ganttproject
[Errno 2] File b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/ganttproject.csv' does not exist: b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/ganttproject.csv'
I am here
Exception occured for  https://github.com/jaydeepw/poly-picker
[Errno 2] File b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/poly-picker.csv' does not exist: b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/poly-picker.csv'
I am here
Exception occured for  https://github.com/testwhat/SmaliEx
[Errno 2] File b'/Users/suvodeepmajumder/Docu

Exception occured for  https://github.com/Glitchfiend/BiomesOPlenty
[Errno 2] File b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/BiomesOPlenty.csv' does not exist: b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/BiomesOPlenty.csv'
I am here
Exception occured for  https://github.com/syndesisio/syndesis
[Errno 2] File b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/syndesis.csv' does not exist: b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/syndesis.csv'
I am here
Exception occured for  https://github.com/lucasferreira/react-native-webview-android
[Errno 2] File b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/react-native-webview-android.csv' does not exist: b'/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/react-native-webview-android.csv'
I am here
Exception occured for  https://github.com/eclipse-ee4j/jersey
[Err

In [21]:
if platform.system() == 'Darwin' or platform.system() == 'Linux':
    data_path = os.getcwd() + '/Test_projects.csv'
else:
    data_path = os.getcwd() + '\\Test_projects.csv'
code_path = os.getcwd()
project_list = pd.read_csv(data_path)
for i in range(project_list.shape[0]):
#     try:
    print("I am here")
    understand_source = []
    last_analyzed = None
    access_token = project_list.loc[i,'access_token']
    repo_owner = project_list.loc[i,'repo_owner']
    source_type = project_list.loc[i,'source_type']
    git_url = project_list.loc[i,'git_url']
    api_base_url = project_list.loc[i,'api_base_url']
    repo_name = project_list.loc[i,'repo_name']
    repo_lang = project_list.loc[i,'lang']
    understand_source.append([1,repo_name,git_url,last_analyzed])
    understand_source_df = pd.DataFrame(understand_source,columns = ['id','name','url','last_analyzed'])
    cas_manager = CAS_Manager(understand_source_df)
    cas_manager.run()
    os.chdir(code_path)
#         git_data = git2data.git2data(access_token,repo_owner,source_type,git_url,api_base_url,repo_name)
#         #git_data = release_mine.git2data(access_token,repo_owner,source_type,git_url,api_base_url,repo_name)
#         git_data.create_data()
#         df_commit = pd.read_pickle(up(code_path) + '/data/commit/' + repo_name + '_commit.pkl')
#         df_commit_guru = pd.read_csv(up(code_path) + '/data/commit_guru/' + repo_name + '.csv')
#         df_commit_guru_subset = df_commit_guru[['commit_hash','contains_bug']].fillna(False)
#         df = pd.merge(df_commit,df_commit_guru_subset, on='commit_hash',how='inner')
#         df.to_pickle(up(code_path) + '/data/commit/' + repo_name + '_commit.pkl')
    get_matrix = git_understand.MetricsGetter(git_url,repo_name,repo_lang)
    matrix = get_matrix.get_defective_pair_metrics()
#     except as e:
#         print("Exception occured for ",project_list.loc[i,'git_url'])
#         print(e)
#         continue

2020-02-23 10:08:33 INFO: Adding repo How-to-Make-a-Computer-Operating-System to work queue for ingesting
2020-02-23 10:08:33 INFO: A worker is starting scan repository: How-to-Make-a-Computer-Operating-System
2020-02-23 10:08:33 INFO: Git cloning repo: How-to-Make-a-Computer-Operating-System


I am here
   id                                     name  \
0   1  How-to-Make-a-Computer-Operating-System   

                                                 url last_analyzed  
0  https://github.com/SamyPesse/How-to-Make-a-Com...          None  
The repo is: How-to-Make-a-Computer-Operating-System
This is the current path: /Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/API_V3/commit_guru/ingester/CASRepos/git/How-to-Make-a-Computer-Operating-System


2020-02-23 10:08:34 INFO: Done cloning.
2020-02-23 10:08:34 INFO: Getting/parsing git commits: How-to-Make-a-Computer-Operating-System
2020-02-23 10:08:34 INFO: Done getting/parsing git commits.
2020-02-23 10:08:34 INFO: Saving commits to the csv...
2020-02-23 10:08:34 INFO: Done saving commits to the csv.
2020-02-23 10:08:34 INFO: A worker finished ingesting repo How-to-Make-a-Computer-Operating-System
2020-02-23 10:08:34 INFO: Adding repo How-to-Make-a-Computer-Operating-System to work queue for analyzing.
2020-02-23 10:08:34 INFO: Worker analyzing repository  How-to-Make-a-Computer-Operating-System
2020-02-23 10:08:34 INFO: Linking 22 new corrective commits for repo How-to-Make-a-Computer-Operating-System
2020-02-23 10:08:34 INFO: Linkage for commit 263cd377a5bdb0ceb1393afa300cc9539fba6025


======================== 2020-02-23 10:08:34.359413 2020-01-24 10:08:34.359397


/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/API_V3/commit_guru/analyzer/git_commit_linker.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  corrective_commit['linked'] = True # mark that we have linked this corrective commit.
/Users/suvodeepmajumder/Conda/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:1015: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc[key] = value
2020-02-23 10:08:34 INFO: Linkage for commit 8556ba031536bbc1cfb560aafac75b1ea7156fcc
2020-02-23 10:08:35 INFO: Linkage for commit e14d57d9352486d1886120ba1fcfc3cbf2cac84d
2020-02-23 10:08:35 INFO: Linkage for commit 834db9ae67d245cb46ae410555da2dedefae

TypeError: __init__() missing 1 required positional argument: 'code_path'

In [ ]:
cas_manager = CAS_Manager()
cas_manager.start()

In [ ]:
for i in range(project_list.shape[0]):
    repo_name = project_list.loc[i,'repo_name']
    df_commit = pd.read_pickle('/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit/' + repo_name + '_commit.pkl')
    df_commit_guru = pd.read_csv('/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru/' + repo_name + '.csv')
    df_commit_guru_subset = df_commit_guru[['commit_hash','contains_bug']].fillna(False)
    df = pd.merge(df_commit,df_commit_guru_subset, on='commit_hash',how='inner')
    df.to_pickle('/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit/' + repo_name + '_commit.pkl')


In [ ]:
if platform.system() == 'Darwin' or platform.system() == 'Linux':
    data_path = os.getcwd() + '/Test_projects.csv'
else:
    data_path = os.getcwd() + '\\Test_projects.csv'

project_list = pd.read_csv(data_path)
print(project_list)

for i in range(project_list.shape[0]):
    try:
        repo_url = project_list.loc[i,'git_url']
        repo_name = project_list.loc[i,'repo_name']
        repo_lang = project_list.loc[i,'lang']
        get_matrix = git_understand.MetricsGetter(repo_url,repo_name,repo_lang)
        #commits = get_matrix.read_commits()
        #matrix = get_matrix.run()
        matrix = get_matrix.get_defective_pair_metrics()
    except ValueError:
        print("Error")

In [ ]:
df_new = pd.merge(matrix,df_commit_guru, on='commit_hash',how='inner')
df_new = df_new.dropna()
df_new.to_csv('/Users/suvodeepmajumder/Documents/AI4SE/Data_Miner/github/data/commit_guru_understand/uhabits.csv',index=False)

In [ ]:
matrix_1 = matrix.drop(['Name','commit_hash'],axis = 1)
matrix_1 = matrix_1.drop_duplicates()
matrix_1.shape

In [ ]:
matrix_1.columns

In [ ]:
os.getcwd()

In [ ]:
if platform.system() == 'Darwin' or platform.system() == 'Linux':
    data_path = os.getcwd() + '/Test_projects.csv'
else:
    data_path = os.getcwd() + '\\Test_projects.csv'
code_path = os.getcwd()
project_list = pd.read_csv(data_path)
for i in range(project_list.shape[0]):
#     try:
    print("I am here")
    understand_source = []
    last_analyzed = None
    access_token = project_list.loc[i,'access_token']
    repo_owner = project_list.loc[i,'repo_owner']
    source_type = project_list.loc[i,'source_type']
    git_url = project_list.loc[i,'git_url']
    api_base_url = project_list.loc[i,'api_base_url']
    repo_name = project_list.loc[i,'repo_name']
    repo_lang = project_list.loc[i,'lang']
    git_client = api_access.git_api_access(access_token,repo_owner,source_type,git_url,api_base_url,repo_name)
    langs = git_client.get_lang()
    project_list.loc[i,'lang'] = langs[0]

In [ ]:
project_list

In [ ]:
project_list.to_csv('projects.csv')

In [ ]:
df1 = pd.read_csv('p_lists.csv')
lists = df1.Projects.values.tolist()

In [ ]:
df2 = pd.read_csv('Test_projects.csv')

In [ ]:
for i in range(df2.shape[0]):
    project = df2.loc[i,'repo_name']
    if project in lists:
        df2.loc[i,'done'] = 1
    else:
        df2.loc[i,'done'] = 0

In [ ]:
df2.to_csv('Test_projects.csv')

In [ ]:
'PushSharp' in lists

In [18]:
project_list_1 = project_list.dropna()

In [23]:
project_list_1

,repo_name,repo_owner,git_url,api_base_url,source_type,access_token,lang,commits,defective,done
0,manifold,manifold-systems,https://github.com/manifold-systems/manifold,http://api.github.com,github_repo,6ff783294993b2f3c1d44506883d62f52ff91f80,Java,1486.0,136.0,0
3,derive4j,derive4j,https://github.com/derive4j/derive4j,http://api.github.com,github_repo,6ff783294993b2f3c1d44506883d62f52ff91f80,Java,251.0,72.0,0
5,nzbhydra2,theotherp,https://github.com/theotherp/nzbhydra2,http://api.github.com,github_repo,6ff783294993b2f3c1d44506883d62f52ff91f80,Java,1752.0,648.0,0
6,polyglot,grpc-ecosystem,https://github.com/grpc-ecosystem/polyglot,http://api.github.com,github_repo,6ff783294993b2f3c1d44506883d62f52ff91f80,Java,287.0,83.0,0
7,progressbar,ctongfei,https://github.com/ctongfei/progressbar,http://api.github.com,github_repo,6ff783294993b2f3c1d44506883d62f52ff91f80,Java,142.0,12.0,0
8,cyclops-integration,aol,https://github.com/aol/cyclops-integration,http://api.github.com,github_repo,6ff783294993b2f3c1d44506883d62f52ff91f80,Java,1299.0,361.0,0
9,RxJavaFX,ReactiveX,https://github.com/ReactiveX/RxJavaFX,http://api.github.com,github_repo,6ff783294993b2f3c1d44506883d62f52ff91f80,Java,327.0,26.0,0
10,Android-Orma,maskarade,https://github.com/maskarade/Android-Orma,http://api.github.com,github_repo,6ff783294993b2f3c1d44506883d62f52ff91f80,Java,1550.0,234.0,0
11,app-icon,dwmkerr,https://github.com/dwmkerr/app-icon,http://api.github.com,github_repo,6ff783294993b2f3c1d44506883d62f52ff91f80,Java,242.0,24.0,0
13,fdb-record-layer,FoundationDB,https://github.com/FoundationDB/fdb-record-layer,http://api.github.com,github_repo,6ff783294993b2f3c1d44506883d62f52ff91f80,Java,936.0,128.0,0


In [31]:
project_list.dropna(inplace=True)
project_list.to_csv('git_list_mod.csv')

In [1]:
import re

In [35]:
fruit_list = ['eapp', 'ban0na', 'pe', 'plum', 'kiwi']
fruit = re.compile('|'.join(fruit_list))
files_changed = ['apple', 'banana', 'peach', 'plum', 'pineapple', 'kiwi']
x = list(filter(fruit.search, list(set(files_changed))))
print(x)

['kiwi', 'pineapple', 'peach', 'plum']


In [41]:
fruit_list = ['eapp', 'ban0na', 'pe', 'plum', 'kiwi']
fruit = re.compile('|'.join(fruit_list))
files_changed = ['apple', 'banana', 'peach', 'plum', 'pineapple', 'kiwi']
x = list(filter(fruit.search, list(set(files_changed))))
print(x)

['kiwi', 'pineapple', 'peach', 'plum']


In [57]:
y = (fruit.search, list(set(files_changed)))

In [58]:
y

(<function Pattern.search(string, pos=0, endpos=9223372036854775807)>,
 ['kiwi', 'ban0na', 'pe', 'plum', 'eapp'])